# UHK SMAP ovalek 01

In [5]:
# install packages
%pip install numpy pandas tensorflow

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 588.3 MB 51 kB/s              
     |████████████████████████████████| 1.7 MB 9.0 MB/s            
     |████████████████████████████████| 4.5 MB 13.5 MB/s            
     |████████████████████████████████| 77 kB 4.9 MB/s             
     |████████████████████████████████| 14.1 MB 8.6 MB/s            
     |████████████████████████████████| 124 kB 10.2 MB/s            
     |████████████████████████████████| 57 kB 6.1 MB/s             
     |████████████████████████████████| 1.1 MB 8.5 MB/s            
     |████████████████████████████████| 2.4 MB 17.2 MB/s            
     |████████████████████████████████| 6.0 MB 16.7 MB/s            
     |████████████████████████████████| 439 kB 15.2 MB/s            
     |████████████████████████████████| 65 kB 5.2 MB/s             
     |████████████████████████████████| 4.8 MB 15.9 MB/s            
     |██████████████████████

In [4]:
import numpy as np
import pandas as pd
import os
print(os.listdir("./data"))

['small', 'parameters_products_color_sanitized.csv', 'parameters_products_pattern_sanitized.csv']


## Build neural network model

In [17]:
# Disable warnings
import warnings
warnings.filterwarnings('ignore')

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf

# Initializes a sequential model
model = Sequential()

# First layer (pictures are 240x340=85000 x 3 colors => 255000)
model.add(Dense(2048, activation='relu', input_shape=(255000, )))

# Second layer
model.add(Dense(256, activation='relu'))

# Output layer (multi-label classification therefore use sigmoid activation function)
model.add(Dense(48, activation='sigmoid'))

model.summary()

2022-12-13 21:41:40.984767: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 2088960000 exceeds 10% of free system memory.
2022-12-13 21:41:41.741512: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 2088960000 exceeds 10% of free system memory.


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 2048)              522242048 
                                                                 
 dense_13 (Dense)            (None, 256)               524544    
                                                                 
 dense_14 (Dense)            (None, 48)                12336     
                                                                 
Total params: 522,778,928
Trainable params: 522,778,928
Non-trainable params: 0
_________________________________________________________________


## Compile a neural network

In [18]:
# multi-label classification therefore use binary_crossentropy
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Use NN for data

In [20]:
# (train_data, train_labels), (test_data, test_labels) = tf.keras.datasets.fashion_mnist.load_data()

# train_data = train_data[(train_labels >= 0) & (train_labels < 3)][0:50].reshape(-1, 28, 28, 1)
# train_labels = train_labels[(train_labels >= 0) & (train_labels < 3)][0:50]
# train_labels = pd.get_dummies(train_labels).to_numpy()

# test_data = test_data[(test_labels >= 0) & (test_labels < 3)][0:10].reshape(-1, 28, 28, 1)
# test_labels = test_labels[(test_labels >= 0) & (test_labels < 3)][0:10]
# test_labels = pd.get_dummies(test_labels).to_numpy()

4422102/4422102 [==============================] - 0s 0us/step
čau
